## Just the dasboard

This notebook contains just the dashboard stuff from the [previous notebook](03_heat_and_trees.ipynb). It might be slightly easier to run since there are no interactions being picked up other than those in the dashboard itself.

In [ ]:
import intake
import pandas as pd

import holoviews as hv
from holoviews import opts
from holoviews.streams import Params
import hvplot.xarray
import hvplot.pandas

import colorcet as cc
import panel as pn

from geoviews.tile_sources import EsriImagery, tile_sources

hv.extension('bokeh', width=90)

description = """
An interactive dashboard for exploring the relationship between 
surface temperature and street trees. Based on Ken Steif's work
summarized in [this blog post](http://urbanspatialanalysis.com/urban-heat-islands-street-trees-in-philadelphia/).
"""

mean_temp_f = intake.open_zarr('./data/mean_temp.zarr').read()
trees_df = pd.read_parquet('./data/trees.parq')

special_cmap = cc.fire[::-1][90:]

temp_p = mean_temp_f\
    .hvplot('x', 'y', title='Surface Temp (F)', 
            geo=True, project=True, 
            cmap=special_cmap, alpha=.5, legend=False, 
            height=500, width=550)\
    .opts(opts.Image(data_aspect=None, clipping_colors={'min': 'transparent', 'max': 'transparent'}))

trees_p = trees_df.hvplot.points('Longitude', 'Latitude', title='Street Tree Density',
                                 geo=True, project=True, datashade=True, dynspread=True,
                                 height=500, width=500, 
                                 legend=False, cmap=cc.kgy[::-1]).opts(data_aspect=None)

start, end = (int(mean_temp_f.temperature.min() - 1), 
              int(mean_temp_f.temperature.max() + 1))
thresh_slider = pn.widgets.RangeSlider(start=start, end=end, value=(start, end), 
                                       step=1, bar_color='#ff0000',
                                       name='Temperature range')
thresh_slider.jslink(temp_p, code={'value': """
color_mapper.low = source.value[0];
color_mapper.high = source.value[1];
"""})

temp_alpha_slider = pn.widgets.FloatSlider(start=0, end=1, value=.5, name='Temperature alpha')
temp_alpha_slider.jslink(temp_p, value='glyph.global_alpha')

trees_toggle = pn.widgets.Toggle(name='Tree Layer', value=True)
trees_toggle.jslink(trees_p, value='glyph_renderer.visible')

trees_alpha_slider = pn.widgets.FloatSlider(start=0, end=1, value=.7, name='Trees alpha')
trees_alpha_slider.jslink(trees_p, value='glyph.global_alpha')

basemap_selector = pn.widgets.Select(name='Tiling Service', value=EsriImagery, options=tile_sources)

def set_tile_source(value):
    return value
    
basemap_p = hv.DynamicMap(set_tile_source, streams=[Params(basemap_selector, ['value'])])

dashboard = pn.Row(
    pn.Column(
        '# Heat and Trees',
        description,
        thresh_slider, 
        temp_alpha_slider,
        basemap_selector,
        trees_toggle,
        trees_alpha_slider),
    temp_p.opts(framewise=False) * trees_p.opts(framewise=False) * basemap_p.opts(framewise=False)
)

dashboard.servable()